In [2]:
%matplotlib inline
%matplotlib nbagg
import pickle
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import prettytensor as pt

In [3]:
rows, cols = 70, 100
img_shape = (rows, cols)
img_flat_size = rows * cols
img_classes = 2
img_channels = 3

phase = pt.Phase.train

In [4]:
resized_folder_path = "/home/openroot/Tanmoy/Working Stuffs/myStuffs/havss-tf/ISIC-2017/resized/"

In [5]:
isic_pickle_file = resized_folder_path + "ISIC.pickle"
with open(isic_pickle_file, "rb") as f:
    save = pickle.load(f)
    train_dataset =  save["train_dataset"]
    train_labels =  save["train_labels"]
    validation_dataset =  save["validation_dataset"]
    validation_labels =  save["validation_labels"]
    test_dataset =  save["test_dataset"]
    test_labels =  save["test_labels"]
    del save
    print("Training Set {0} {1}".format(train_dataset.shape, train_labels.shape))
    print("Validation Set {0} {1}".format(validation_dataset.shape, validation_labels.shape))    
    print("Test Set {0} {1}".format(test_dataset.shape, test_labels.shape))
print test_labels[0:10, :]
test_dataset_info = np.sum(test_labels[:, :], axis = 0)
print("Test Dataset Info : [1, 0] = {0}, [0, 1] = {1}".format(test_dataset_info[0], test_dataset_info[1]))
train_dataset_info = np.sum(validation_labels[:, :], axis = 0)
print("Validation Dataset Info : [1, 0] = {0}, [0, 1] = {1}".format(train_dataset_info[0], train_dataset_info[1]))
train_dataset_info = np.sum(train_labels[:, :], axis = 0)
print("Train Dataset Info : [1, 0] = {0}, [0, 1] = {1}".format(train_dataset_info[0], train_dataset_info[1]))

Training Set (37904, 70, 100, 3) (37904, 2)
Validation Set (6152, 70, 100, 3) (6152, 2)
Test Set (6152, 70, 100, 3) (6152, 2)
[[ 0.  1.]
 [ 0.  1.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 0.  1.]
 [ 1.  0.]
 [ 1.  0.]
 [ 0.  1.]]
Test Dataset Info : [1, 0] = 2786.0, [0, 1] = 3366.0
Validation Dataset Info : [1, 0] = 2522.0, [0, 1] = 3630.0
Train Dataset Info : [1, 0] = 20612.0, [0, 1] = 17292.0


In [6]:
# input_x = tf.placeholder(tf.float32, [None, img_flat_size], name = "input_x") 
input_layer = tf.placeholder(tf.float32, [None, rows, cols, img_channels], name = "input_layer")
y_true = tf.placeholder(tf.float32, [None, img_classes], name = "y_true")
y_true_cls = tf.argmax(y_true, dimension = 1)

In [7]:
x_pretty = pt.wrap(input_layer)
with pt.defaults_scope(activation_fn = tf.nn.relu):
    y_pred, loss = x_pretty.\
        conv2d(kernel = 5, depth = 16, name = "conv_layer_1").\
        max_pool(kernel = 2, stride = 2).\
        dropout(keep_prob=0.8, phase=phase).\
        conv2d(kernel = 5, depth = 32, name = "conv_layer_2").\
        max_pool(kernel = 2, stride = 2).\
        dropout(keep_prob=0.8, phase=phase).\
        conv2d(kernel = 5, depth = 64, name = "conv_layer_3").\
        max_pool(kernel = 2, stride = 2).\
        flatten().\
        fully_connected(size = 256, name = "fc_layer_2", activation_fn = tf.nn.relu).\
        softmax_classifier(num_classes = img_classes, labels = y_true)

In [8]:
global_step = tf.Variable(0, trainable=False)
learning_rate = tf.train.exponential_decay(0.1, global_step, train_labels.shape[0], 0.95, staircase=True)

In [9]:
optimizer = tf.train.AdamOptimizer(learning_rate = 0.001).minimize(loss)

In [10]:
y_pred_cls = tf.argmax(y_pred, dimension = 1)

In [11]:
correct_pred = tf.equal(y_true_cls, y_pred_cls)

In [12]:
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [13]:
train_batch_size = 5
prev_train_acc = 0
prev_train_acc_step = 0
min_train_acc_step = 1000
offset = 0

In [ ]:
num_steps = 94720
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    print("Variable Initialized")
    for step in range(num_steps):
        offset = (step * train_batch_size) % (train_labels.shape[0] - train_batch_size)
#         print offset
        batch_train = train_dataset[offset:(offset + train_batch_size), :]
        batch_labels = train_labels[offset:(offset + train_batch_size), :]
        feed_dict_train = {
            input_layer: batch_train,
            y_true: batch_labels
        }
        
        _, l = sess.run([optimizer, loss], feed_dict = feed_dict_train)
        print("Step : {0} - Loss : {1}".format(step, l))
        

        if step % 200 :
            feed_dict_valid = {
                input_layer: validation_dataset,
                y_true: validation_labels
            }

            valid_acc = sess.run([accuracy], feed_dict = feed_dict_valid)
            print("Step : {0} - Validation Accuracy : {1} ".format(step, train_loss))
            
    phase = pt.Phase.test    
    feed_dict_test = {
        input_layer: test_dataset,
        y_true: test_labels
    }
    true_value_class, acc, pred_value, pred_value_class = sess.run([y_true_cls, accuracy, y_pred, y_pred_cls], feed_dict = feed_dict_test)
    print("Accuracy {:.2%}".format(acc))
    print("True Value Class")
    print true_value_class[0:10]
    print("Pred Value")
    print pred_value[0:20]
    print("Pred Value Class")
    print pred_value_class

Variable Initialized
0
Loss : 78.988243103
5
Loss : 20.2382469177
10
Loss : 0.000680456287228
15
Loss : 4.23762464523
20
Loss : 15.5507183075
25
Loss : 0.514702737331
30
Loss : 2.75313830376
35
Loss : 1038.46276855
40
Loss : 11.5108823776
45
Loss : 9.52188682556
50
Loss : 32.706577301
55
Loss : 21.9293880463
60
Loss : 80.6869125366
65
Loss : 30.5660438538
70
Loss : 50.7009468079
75
Loss : 93.2705078125
80
Loss : 12.719086647
85
Loss : 75.0146408081
90
Loss : 20.0377998352
95
Loss : 5.61265230179
100
Loss : 32.5840530396
105
Loss : 86.4105529785
110
Loss : 45.6695976257
115
Loss : 25.2432441711
120
Loss : 2.81038737297
125
Loss : 22.9047203064
130
Loss : 14.0278835297
135
Loss : 1.8533872366
140
Loss : 7.15255623618e-08
145
Loss : 20.3479862213
150
Loss : 12.92537117
155
Loss : 12.8175640106
160
Loss : 7.02047109604
165
Loss : 15.0022993088
170
Loss : 3.91904973984
175
Loss : 7.16133880615
180
Loss : 10.5501880646
185
Loss : 4.84490251541
190
Loss : 1.4270362854
195
Loss : 1.20854985714

Loss : 263.593444824
1600
Loss : 1.31528544426
1605
Loss : 0.488919913769
1610
Loss : 0.725866675377
1615
Loss : 0.805526912212
1620
Loss : 0.776932358742
1625
Loss : 0.794943094254
1630
Loss : 1.05928349495
1635
Loss : 0.752769768238
1640
Loss : 0.377065807581
1645
Loss : 0.657449066639
1650
Loss : 0.801725029945
1655
Loss : 0.782132983208
1660
Loss : 0.777432560921
1665
Loss : 0.899295628071
1670
Loss : 1.00314307213
1675
Loss : 0.713771641254
1680
Loss : 0.777764439583
1685
Loss : 0.990886390209
1690
Loss : 0.81845921278
1695
Loss : 0.604293227196
1700
Loss : 0.677856445312
1705
Loss : 0.576002299786
1710
Loss : 0.75092202425
1715
Loss : 0.750534653664
1720
Loss : 0.621950924397
1725
Loss : 0.634402394295
1730
Loss : 0.646581292152
1735
Loss : 0.627069473267
1740
Loss : 0.764342188835
1745
Loss : 0.764180541039
1750
Loss : 28.3629722595
1755
Loss : 29.5572929382
1760
Loss : 0.666206717491
1765
Loss : 0.63284933567
1770
Loss : 0.769238114357
1775
Loss : 0.637468457222
1780
Loss : 0.5

Loss : 0.598323881626
3135
Loss : 0.685343801975
3140
Loss : 0.558935046196
3145
Loss : 0.586631834507
3150
Loss : 1.51124322414
3155
Loss : 0.652960598469
3160
Loss : 0.600026726723
3165
Loss : 0.738811135292
3170
Loss : 0.667407631874
3175
Loss : 0.622364521027
3180
Loss : 1.1827532053
3185
Loss : 0.67414700985
3190
Loss : 0.764570355415
3195
Loss : 0.745723605156
3200
Loss : 0.491235882044
3205
Loss : 0.70250838995
3210
Loss : 0.617982745171
3215
Loss : 0.779265999794
3220
Loss : 0.51623904705
3225
Loss : 0.526448488235
3230
Loss : 0.58146238327
3235
Loss : 0.611197113991
3240
Loss : 1.94779622555
3245
Loss : 0.641013681889
3250
Loss : 0.658420443535
3255
Loss : 0.750758171082
3260
Loss : 0.580400466919
3265
Loss : 0.594485282898
3270
Loss : 0.729495346546
3275
Loss : 0.660337865353
3280
Loss : 0.560722708702
3285
Loss : 0.654514789581
3290
Loss : 0.605770409107
3295
Loss : 0.602716743946
3300
Loss : 0.660006821156
3305
Loss : 0.654850780964
3310
Loss : 0.509805381298
3315
Loss : 0.

Loss : 0.678436636925
4665
Loss : 0.696165084839
4670
Loss : 0.705909669399
4675
Loss : 0.619144558907
4680
Loss : 0.651335060596
4685
Loss : 0.646637797356
4690
Loss : 0.571871876717
4695
Loss : 0.60902005434
4700
Loss : 0.799414992332
4705
Loss : 0.583214879036
4710
Loss : 0.694336295128
4715
Loss : 0.577610492706
4720
Loss : 0.537245869637
4725
Loss : 0.659769952297
4730
Loss : 0.714155673981
4735
Loss : 0.671301722527
4740
Loss : 0.835183739662
4745
Loss : 0.701287865639
4750
Loss : 0.603986918926
4755
Loss : 0.637608408928
4760
Loss : 0.575920701027
4765
Loss : 0.419991344213
4770
Loss : 0.394942939281
4775
Loss : 0.826372146606
4780
Loss : 0.6227902174
4785
Loss : 0.816501259804
4790
Loss : 0.395577728748
4795
Loss : 0.899642348289
4800
Loss : 0.638675093651
4805
Loss : 0.576384663582
4810
Loss : 0.769519686699
4815
Loss : 0.715070843697
4820
Loss : 0.729340553284
4825
Loss : 0.644828021526
4830
Loss : 0.608867645264
4835
Loss : 0.631082773209
4840
Loss : 0.648863911629
4845
Loss

Loss : 0.68503677845
6195
Loss : 0.58588218689
6200
Loss : 0.66804265976
6205
Loss : 0.696733117104
6210
Loss : 0.685422897339
6215
Loss : 0.622800946236
6220
Loss : 0.659222900867
6225
Loss : 0.657586455345
6230
Loss : 0.44247302413
6235
Loss : 0.67712777853
6240
Loss : 0.658299267292
6245
Loss : 0.60050290823
6250
Loss : 0.586775898933
6255
Loss : 0.370509803295
6260
Loss : 0.574922382832
6265
Loss : 0.616319835186
6270
Loss : 0.568540930748
6275
Loss : 0.596191823483
6280
Loss : 0.521069705486
6285
Loss : 0.590849637985
6290
Loss : 0.729782283306
6295
Loss : 0.53870600462
6300
Loss : 0.531834304333
6305
Loss : 0.735073685646
6310
Loss : 0.697585940361
6315
Loss : 0.569387793541
6320
Loss : 0.549635827541
6325
Loss : 0.500872015953
6330
Loss : 6.22504711151
6335
Loss : 0.49968713522
6340
Loss : 0.647619128227
6345
Loss : 0.468724429607
6350
Loss : 0.522872149944
6355
Loss : 0.720783352852
6360
Loss : 0.646134734154
6365
Loss : 0.606859564781
6370
Loss : 0.768835008144
6375
Loss : 0.6

...

Loss : 0.148600891232
14261
Loss : 0.69316971302
14266
Loss : 0.00700625404716
14271
Loss : 0.00541934510693
14276
Loss : 0.0836814343929
14281
Loss : 0.0801624655724
14286
Loss : 0.101058460772
14291
Loss : 0.183210045099
14296
Loss : 0.0533084645867
14301
Loss : 0.19638517499
14306
Loss : 0.361881494522
14311
Loss : 0.117878399789
14316
Loss : 0.599284946918
14321
Loss : 0.348204851151
14326
Loss : 0.441543579102
14331
Loss : 0.204274490476
14336
Loss : 0.0620575733483
14341
Loss : 0.379129320383
14346
Loss : 0.296645432711
14351
Loss : 0.256517231464
14356
Loss : 0.0687496140599
14361
Loss : 0.0308383516967
14366
Loss : 0.194653645158
14371
Loss : 0.0963763073087
14376
Loss : 0.195769876242
14381
Loss : 0.0656704455614
14386
Loss : 0.0330699458718
14391
Loss : 0.2690076828
14396
Loss : 0.0504023917019
14401
Loss : 0.129202112556
14406
Loss : 0.243120834231
14411
Loss : 0.360057473183
14416
Loss : 0.542227804661
14421
Loss : 0.0715329423547
14426
Loss : 0.0399155132473
14431
Loss : 0

Loss : 0.00693307677284
15716
Loss : 0.0564090423286
15721
Loss : 0.28909894824
15726
Loss : 0.178489074111
15731
Loss : 0.305505216122
15736
Loss : 0.00809833034873
15741
Loss : 0.168079078197
15746
Loss : 0.680812239647
15751
Loss : 0.307535886765
15756
Loss : 0.164309799671
15761
Loss : 0.186419218779
15766
Loss : 0.0938427746296
15771
Loss : 0.169442877173
15776
Loss : 0.434127509594
15781
Loss : 0.372624456882
15786
Loss : 0.158015877008
15791
Loss : 0.516013801098
15796
Loss : 0.00991825945675
15801
Loss : 0.0643163248897
15806
Loss : 0.101348660886
15811
Loss : 0.0703274160624
15816
Loss : 0.126286178827
15821
Loss : 1.14809679985
15826
Loss : 0.137198656797
15831
Loss : 0.604905843735
15836
Loss : 0.0279294494539
15841
Loss : 0.258536398411
15846
Loss : 0.166997492313
15851
Loss : 0.250462114811
15856
Loss : 0.475817382336
15861
Loss : 0.072453148663
15866
Loss : 0.0915885791183
15871
Loss : 0.0271710101515
15876
Loss : 0.256353855133
15881
Loss : 0.0497104525566
15886
Loss : 0

Loss : 0.153327941895
17161
Loss : 0.056381482631
17166
Loss : 0.0943841487169
17171
Loss : 0.0465720146894
17176
Loss : 0.00440211687237
17181
Loss : 0.0632826238871
17186
Loss : 0.00120955461171
17191
Loss : 0.174153983593
17196
Loss : 0.360861331224
17201
Loss : 0.329153776169
17206
Loss : 0.0362606085837
17211
Loss : 0.00150104449131
17216
Loss : 0.00451423320919
17221
Loss : 0.196458250284
17226
Loss : 0.413844019175
17231
Loss : 0.064229786396
17236
Loss : 0.0626899823546
17241
Loss : 0.00627295672894
17246
Loss : 0.387239009142
17251
Loss : 0.740988075733
17256
Loss : 0.0216885246336
17261
Loss : 0.0220988467336
17266
Loss : 0.0950411558151
17271
Loss : 0.480500459671
17276
Loss : 0.00150756759103
17281
Loss : 0.0418646149337
17286
Loss : 0.078441850841
17291
Loss : 0.236854314804
17296
Loss : 0.0161565858871
17301
Loss : 0.0248828548938
17306
Loss : 0.0862223058939
17311
Loss : 0.0239793118089
17316
Loss : 0.285161077976
17321
Loss : 0.192036032677
17326
Loss : 0.167567566037
1

Loss : 0.0787561237812
18601
Loss : 0.159465119243
18606
Loss : 0.169429138303
18611
Loss : 0.254756122828
18616
Loss : 0.915034115314
18621
Loss : 0.0759300664067
18626
Loss : 0.0108317593113
18631
Loss : 0.0601062066853
18636
Loss : 0.199223875999
18641
Loss : 0.0418149381876
18646
Loss : 0.000742622825783
18651
Loss : 0.0781391113997
18656
Loss : 0.534435331821
18661
Loss : 0.247489482164
18666
Loss : 0.0939546898007
18671
Loss : 0.166146203876
18676
Loss : 0.159468263388
18681
Loss : 0.197107389569
18686
Loss : 0.000753025349695
18691
Loss : 0.618019402027
18696
Loss : 0.383752763271
18701
Loss : 1.03247010708
18706
Loss : 0.0936934575438
18711
Loss : 0.166566893458
18716
Loss : 0.166783243418
18721
Loss : 0.0471536554396
18726
Loss : 0.566086411476
18731
Loss : 0.0426363237202
18736
Loss : 0.0623993575573
18741
Loss : 0.0144589934498
18746
Loss : 0.0211492814124
18751
Loss : 0.0367548018694
18756
Loss : 0.110707782209
18761
Loss : 0.341841369867
18766
Loss : 0.000950861081947
1877

Loss : 0.165005892515
20041
Loss : 0.256996780634
20046
Loss : 0.689332127571
20051
Loss : 0.0295737572014
20056
Loss : 0.111157402396
20061
Loss : 0.00297864479944
20066
Loss : 0.643712162971
20071
Loss : 0.114740371704
20076
Loss : 0.229319006205
20081
Loss : 0.185757622123
20086
Loss : 0.00235116155818
20091
Loss : 0.000301440682961
20096
Loss : 0.474702507257
20101
Loss : 0.160059899092
20106
Loss : 0.342623144388
20111
Loss : 0.219278335571
20116
Loss : 0.0160295125097
20121
Loss : 0.270338386297
20126
Loss : 0.0478604733944
20131
Loss : 0.143554776907
20136
Loss : 0.287535756826
20141
Loss : 0.0265488438308
20146
Loss : 0.276073276997
20151
Loss : 0.152241289616
20156
Loss : 0.0048712817952
20161
Loss : 0.11871547997
20166
Loss : 0.00269045028836
20171
Loss : 0.0591152086854
20176
Loss : 0.00206856522709
20181
Loss : 0.0902713909745
20186
Loss : 0.0658520981669
20191
Loss : 0.59098136425
20196
Loss : 0.137370273471
20201
Loss : 0.0109422244132
20206
Loss : 0.00119651737623
20211


In [ ]:
def runModel(epoch):
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        print("Variables Initialized")
        total_data_len = train_labels.shape[0]
        print total_data_len
        num_step = total_data_len / train_batch_size
        print("Batch Size : {0}".format(train_batch_size))
        print("Starting Training")
        for i in range(epoch):
            print("Epoch - {0} ".format(i))
            for step in range(num_step):
                offset = (step * train_batch_size) % (total_data_len - train_batch_size)
#                 print offset
                batch_train = train_dataset[offset:(offset + train_batch_size), :, :, :]
                batch_labels = train_labels[offset:(offset + train_batch_size), :]
                feed_dict_train = {
                    input_layer: batch_train,
                    y_true: batch_labels
#                     global_step: step * train_batch_size
                }

                _, train_loss = sess.run([optimizer, loss], feed_dict = feed_dict_train)
                print("Step : {0} - Training Loss : {1}".format(step, train_loss))
                
                if step % 200 :
                    feed_dict_valid = {
                        input_layer: validation_dataset,
                        y_true: validation_labels
                    }
                    
                    valid_acc = sess.run([accuracy], feed_dict = feed_dict_valid)
                    print("Step : {0} - Validation Accuracy : {1} ".format(step, train_loss))
                    
                    if valid_acc > prev_train_acc:
                        print("Imporvement Found : {0}, Step Difference : {1}".format(valid_acc - prev_train_acc, step - prev_train_acc_step))
                        prev_train_acc_step = step
                        prev_train_acc = valid_acc
                    else:
                        if prev_train_acc_step - step > min_train_acc_step:
                            print("No Improvement Found : Step Difference : {0}".format(step - prev_train_acc_step))
                            print("Therefore Ending Training")
                            break
        print("Training Completed!!!")
        print("Staring Testing Phase")
        phase = pt.Phase.test    
        feed_dict_test = {
            input_layer: test_dataset,
            y_true: test_labels
        }      
        test_acc = sess.run([accuracy], feed_dict = feed_dict_test)
        print("Testing Accuracy : {0}".format(test_acc))

In [ ]:
# runModel(10)